In [1]:
import torch
import cupy as cp
import time
from cuml.svm import SVC
from cuml.metrics import accuracy_score


In [3]:
model = 'activations_pythia_410m/data_subset/model-activations/EleutherAI_pythia-410m_11L_6.pt'
sae = 'activations_pythia_410m/data_subset/sae-activations/EleutherAI_sae-pythia-410m-65k_11L_6.pt'


checkpoint_model = torch.load(model, map_location=torch.device('cuda:0'), weights_only=False)
print(len(checkpoint_model))
print(type(checkpoint_model))
print(checkpoint_model[3])
print(checkpoint_model[4])
print(checkpoint_model[5])

print('_'*25)
checkpoint_sae = torch.load(sae, map_location=torch.device('cuda:0'), weights_only=False)
checkpoint_sae = [c.to_dense() if c is not None else None for c in checkpoint_sae]
print(len(checkpoint_sae))
print(type(checkpoint_sae))
print(checkpoint_sae[3])
print(checkpoint_sae[4])
print(checkpoint_sae[5])

1024
<class 'list'>
tensor([[ 0.0492,  0.5470,  0.1375,  ...,  0.2262, -0.2788, -0.2633],
        [-0.2356,  0.0811, -0.1012,  ..., -0.1610,  0.1629,  0.0760],
        [ 0.2019, -0.0635, -0.0820,  ...,  0.2694, -0.0834,  0.0086],
        ...,
        [ 0.2713,  0.3616, -0.2879,  ...,  0.0356,  0.1065, -0.1837],
        [-0.0131,  0.0603, -0.1651,  ...,  0.3494,  0.0027, -0.3750],
        [ 0.0200,  0.6732, -0.4007,  ...,  0.5561,  0.2307, -0.4278]],
       device='cuda:0')
tensor([[-0.3050, -0.0791, -0.2149,  ...,  0.1548, -0.1444, -0.3251],
        [-0.0615,  0.1040, -0.1930,  ...,  0.3543, -0.1184, -0.0146],
        [-0.1862,  0.0594,  0.1303,  ...,  0.2660, -0.2469,  0.2455],
        ...,
        [-0.5425,  0.0558, -0.0265,  ...,  0.1966, -0.0360, -0.3869],
        [-0.0101, -0.0076,  0.4090,  ...,  0.2384,  0.1096,  0.3491],
        [-0.1762,  0.1220, -0.0825,  ...,  0.0574, -0.0748,  0.0346]],
       device='cuda:0')
tensor([[-0.3260, -0.3580,  0.0437,  ...,  0.0821, -0.0599,  0.1

In [7]:
torch.cat(checkpoint_model, dim=0).shape

torch.Size([55275, 1024])

In [6]:
checkpoint_model[1].shape

torch.Size([48, 1024])

In [7]:
# ---------------------------
# 2. Convert PyTorch tensors to CuPy arrays using DLPack (zero-copy)
# ---------------------------
X_cupy = cp.from_dlpack(torch.utils.dlpack.to_dlpack(X_torch))
y_cupy = cp.from_dlpack(torch.utils.dlpack.to_dlpack(y_torch))

In [8]:
type(X_cupy)

cupy.ndarray

In [9]:
X_cupy.shape

(5000, 65000)

In [11]:
X_cupy[0,:].shape

(65000,)

In [21]:
# ---------------------------
# 3. Initialize and fit cuML's SVC with a linear kernel
# ---------------------------
start = time.time()
svc = SVC(kernel='linear', max_iter=1000)


svc.fit(X_cupy, y_cupy)
end = time.time()
print("cuML SVC training time: {:.4f} seconds".format(end - start))

cuML SVC training time: 0.9053 seconds


In [20]:
# ---------------------------
# 4. Predict and compute accuracy on the same data (for demonstration)
# ---------------------------
pred = svc.predict(X_cupy)
acc = accuracy_score(y_cupy, pred)
print("Test accuracy: {:.4f}".format(acc))

Test accuracy: 1.0000
